In [1]:
### Imports
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import gensim
import scipy.sparse as sparse
# Set seaborn color palette
colors = sns.color_palette("pastel")
# Import for checking th python version
import struct
%matplotlib inline
# Surprise related Imports
from surprise import SVD
from surprise import Dataset
from surprise import evaluate, print_perf
import time

In [2]:
# Loading Orders_Users Dataset
df_orders = pd.read_csv("../data/orders.csv")
df_orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [3]:
# Loading Orders_Products Dataset
df_order_products = pd.read_csv("../data/order_products__prior.csv")
df_products = pd.read_csv("../data/products.csv").set_index('product_id')

In [4]:
df_order_products.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [5]:
df_products.head()

,product_name,aisle_id,department_id
product_id,,,
1,Chocolate Sandwich Cookies,61,19
2,All-Seasons Salt,104,13
3,Robust Golden Unsweetened Oolong Tea,94,7
4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
5,Green Chile Anytime Sauce,5,13


In [6]:
# Remove this '.head(100000)' part to run for complete Dataset
df_order_products_sample=df_order_products.head(100000)

df_order_products_sample["product_id"] = df_order_products_sample["product_id"].astype(str)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [7]:
df_order_products_sample.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [8]:
# Saving list of products per oredr ID
df_order_products_sample = df_order_products_sample.groupby("order_id").apply(lambda order: order['product_id'].tolist())

In [9]:
df_order_products_sample.head()

order_id
2    [33120, 28985, 9327, 45918, 30035, 17794, 4014...
3    [33754, 24838, 17704, 21903, 17668, 46667, 174...
4    [46842, 26434, 39758, 27761, 10054, 21351, 225...
5    [13176, 15005, 47329, 27966, 23909, 48370, 132...
6                                [40462, 15873, 41897]
dtype: object

In [13]:
# taking the longest order length
longest = np.max(df_order_products_sample.apply(len))

In [14]:
# Making Orders to sentences of word2Vec
sentences = df_order_products_sample.values

In [15]:
# Model on WordtoVec for Similar Prodcuts
model= gensim.models.Word2Vec(sentences, size=100, window=longest, min_count=2, workers=4)

In [16]:
#vocabulary of all the keys:
vocab = list(model.wv.vocab.keys())

In [17]:
# Get Top N similar Products to a product
def get_similar_products(product_id,top_n):
    suggestions = model.most_similar(positive=[vocab[product_id]], topn=top_n)
    output = list()
    for suggestion in suggestions:
        output.append((int(suggestion[0]),df_products.loc[int(suggestion[0])]['product_name'],suggestion[1]))
    return output

In [20]:
#Printing k similar items for n random Products:
def print_k_suggestions_for_n_prodcuts(n,k):
    for i in range(n):
        print("Product is : ",df_products.loc[int(vocab[i])]['product_name'])
        print("related products are : ")
        suggestions=get_similar_products(n,k)
        print([suggestion[1] for suggestion in suggestions])

In [22]:
print_k_suggestions_for_n_prodcuts(2,5)

Product is :  Organic Whole Chocolate Milk
related products are : 
['Spinach and Arugula Salad', 'Super Puffs Spinach & Apple Fruit & Veggie Grain Puffs', 'Mellow White Miso', 'Plain Low Fat Australian Style Yogurt', 'Sesame Bagels']
Product is :  Green Seedless Grapes
related products are : 
['Spinach and Arugula Salad', 'Super Puffs Spinach & Apple Fruit & Veggie Grain Puffs', 'Mellow White Miso', 'Plain Low Fat Australian Style Yogurt', 'Sesame Bagels']
